In [11]:
import torch

def round_pass(x):
    y = x.round()
    y_grad = x
    return y.detach() - y_grad.detach() + y_grad

class Quantizer():
    def __init__(self, N_bits: int, type: str = "per_tensor",  signed: bool = True, symmetric: bool = True):
        super().__init__()
            
        self.N_bits = N_bits
        self.signed = signed
        self.symmetric = symmetric
        self.q_type = type
        # self.eps = torch.iinfo(dtype).eps
        # self.minimum_range = torch.iinfo(dtype).eps
        if self.N_bits is None:
            return 

        if self.signed:
            self.Qn = - 2 ** (self.N_bits - 1)
            self.Qp = 2 ** (self.N_bits - 1) - 1
        else:
            self.Qn = 0
            self.Qp = 2 ** self.N_bits - 1

    def __call__(self, x):  
        return self.forward(x)

    def forward(self, x): 
        if self.N_bits is None:
            return x, 1

        if self.symmetric:
            if self.q_type == 'per_tensor': 
                max_x = x.abs().max().detach()
            elif self.q_type == 'per_token': 
                max_x = x.abs().amax(dim=-1, keepdim=True).detach()
            elif self.q_type == 'per_channel': 
                max_x = x.abs().amax(dim=0, keepdim=True).detach()

            print(max_x)
            scale = max_x / self.Qp
            x = x / scale 
            x = round_pass(x.clamp_(self.Qn, self.Qp)) 
            
        else: #Asymmetric
            min_x = x.min().detach()
            max_x = x.max().detach()
            range_x = (max_x - min_x).detach().clamp_(min=self.minimum_range)
            scale = range_x / (self.Qp - self.Qn)

            zero_point = torch.round((min_x / scale) - self.Qn)

            x = (x / scale) + zero_point
            x = round_pass(x.clamp_(self.Qn, self.Qp))

        return x, scale

quantizer = Quantizer (4, 'per_channel')

x = torch.rand(3,5) 
print("x",x)

q_x, s_x = quantizer(x)
print("q_x", q_x)
print("s_x", s_x)

x tensor([[0.5318, 0.7585, 0.7211, 0.5480, 0.9295],
        [0.0402, 0.1398, 0.4303, 0.4139, 0.3432],
        [0.3358, 0.5768, 0.3559, 0.3142, 0.0769]])
tensor([[0.5318, 0.7585, 0.7211, 0.5480, 0.9295]])
q_x tensor([[7., 7., 7., 7., 7.],
        [1., 1., 4., 5., 3.],
        [4., 5., 3., 4., 1.]])
s_x tensor([[0.0760, 0.1084, 0.1030, 0.0783, 0.1328]])


In [12]:
import torch
import numpy as np


def process_tensor(x,iteration, layer: str):
    # x: Tensor of shape [BS, sequence_length, channel_dim]
    BS = x.shape[0]
    sequence_length = x.shape[1]
    channel_dim = x.shape[2]
    num_elements = sequence_length * channel_dim

    top1_values = []
    top3_values = []
    top3_indices = []
    top1_percent_values = []
    median_values = []

    for i in range(BS):
        sample = x[i]  # Shape [sequence_length, channel_dim]
        flattened = sample.view(-1)  # Flatten to 1D tensor of size sequence_length * channel_dim
        sorted_values, sorted_indices = torch.sort(flattened, descending=True)

        # Top1
        top1_value = sorted_values[0].item()
        top1_values.append(top1_value)


        # Top3
        top3_values_i = sorted_values[:3].cpu().numpy()
        top3_indices_flat = sorted_indices[:3].cpu().numpy()
        # Map back to 2D indices
        print("top3_indices_flat",top3_indices_flat )
        print("channel_dim",channel_dim )

        
        top3_rows = top3_indices_flat // channel_dim
        top3_cols = top3_indices_flat % channel_dim
        top3_indices_i = list(zip(top3_rows, top3_cols))
        top3_values.append(top3_values_i)
        top3_indices.append(top3_indices_i)

        # Top1% elements
        top1_percent_count = max(1, int(np.ceil(num_elements * 0.01)))
        top1_percent = sorted_values[:top1_percent_count].cpu().numpy()
        top1_percent_values.append(top1_percent)

        # Median value
        median = torch.median(flattened).item()
        median_values.append(median)

    # Compute mean and standard deviation for each quantity
    top1_mean = np.mean(top1_values)
    top1_std = np.std(top1_values)

    top3_means = [np.mean(vals) for vals in top3_values]
    top3_mean = np.mean(top3_means)
    top3_std = np.std(top3_means)

    top1_percent_mean = np.mean([np.mean(vals) for vals in top1_percent_values])
    top1_percent_std = np.std([np.mean(vals) for vals in top1_percent_values])

    median_mean = np.mean(median_values)
    median_std = np.std(median_values)
    epoch = iteration//1252
    # Save results to a file
    with open('probe_result.txt', 'a') as f:
        f.write('Epoch: {}\n'.format(epoch))
        f.write('Layer info: {}\n'.format(layer))

        f.write('Top1 Mean: {:.4f}, Top1 Std: {:.4f}\n'.format(top1_mean, top1_std))
        f.write('Top3 Mean: {:.4f}, Top3 Std: {:.4f}\n'.format(top3_mean, top3_std))
        f.write('Top1% Mean: {:.4f}, Top1% Std: {:.4f}\n'.format(top1_percent_mean, top1_percent_std))
        f.write('Median Mean: {:.4f}, Median Std: {:.4f}\n'.format(median_mean, median_std))

        f.write('\nTop 3 values and their positions for each sample:\n')
        for i in range(BS):
            f.write('Sample {}:\n'.format(i))
            for rank, (value, (row, col)) in enumerate(zip(top3_values[i], top3_indices[i]), 1):
                f.write('  Rank {}: Value {:.4f} at position (sequence index {}, channel index {})\n'.format(
                    rank, value, row, col))
        f.write('\n' + '-' * 50 + '\n')  # Separator between calls




# x = torch.rand(256,197,384)
x = torch.tensor([[[1, 10, 5], 
     [4, 5, 7]]])
process_tensor(x, 15, 'dd')


top3_indices_flat [1 5 2]
channel_dim 3


In [16]:
import torch
import numpy as np
import csv


def process_tensor(x,iteration, layer: str):
    # x: Tensor of shape [BS, sequence_length, channel_dim]
    BS = x.shape[0]
    sequence_length = x.shape[1]
    channel_dim = x.shape[2]
    num_elements = sequence_length * channel_dim

    top1_values = []
    top3_values = []
    top3_indices = []
    top1_percent_values = []
    median_values = []

    for i in range(BS):
        sample = x[i]  # Shape [sequence_length, channel_dim]
        flattened = sample.view(-1)  # Flatten to 1D tensor of size sequence_length * channel_dim
        sorted_values, sorted_indices = torch.sort(flattened, descending=True)

        # Top1
        top1_value = sorted_values[0].item()
        top1_values.append(top1_value)


        # Top3
        top3_values_i = sorted_values[:3].cpu().numpy()
        top3_indices_flat = sorted_indices[:3].cpu().numpy()
        # Map back to 2D indices
        print("top3_indices_flat",top3_indices_flat )
        print("channel_dim",channel_dim )

        
        top3_rows = top3_indices_flat // channel_dim
        top3_cols = top3_indices_flat % channel_dim
        top3_indices_i = list(zip(top3_rows, top3_cols))
        top3_values.append(top3_values_i)
        top3_indices.append(top3_indices_i)

        # Top1% elements
        top1_percent_count = max(1, int(np.ceil(num_elements * 0.01)))
        top1_percent = sorted_values[:top1_percent_count].cpu().numpy()
        top1_percent_values.append(top1_percent)

        # Median value
        median = torch.median(flattened).item()
        median_values.append(median)

    # Compute mean and standard deviation for each quantity
    top1_mean = np.mean(top1_values)
    top1_std = np.std(top1_values)

    top3_means = [np.mean(vals) for vals in top3_values]
    top3_mean = np.mean(top3_means)
    top3_std = np.std(top3_means)

    top1_percent_mean = np.mean([np.mean(vals) for vals in top1_percent_values])
    top1_percent_std = np.std([np.mean(vals) for vals in top1_percent_values])

    median_mean = np.mean(median_values)
    median_std = np.std(median_values)
    epoch = iteration//1252

    # 통계 데이터를 probe_result.csv에 기록
    with open('/home/shkim/QT_DeiT_small/probe_result.csv', 'a', newline='') as f:
        writer = csv.writer(f)
        if f.tell() == 0:  # 파일이 비어 있으면 헤더 추가
            writer.writerow(['Epoch', 'Layer', 'Top1 Mean', 'Top1 Std', 'Top3 Mean', 'Top3 Std', 
                             'Top1% Mean', 'Top1% Std', 'Median Mean', 'Median Std'])
        writer.writerow([epoch, layer, top1_mean, top1_std, top3_mean, top3_std, 
                         top1_percent_mean, top1_percent_std, median_mean, median_std])

    # 각 배치의 Top3 인덱스를 top3_indices.csv에 기록
    with open('/home/shkim/QT_DeiT_small/top3_indices.csv', 'a', newline='') as f:
        writer = csv.writer(f)
        if f.tell() == 0:  # 파일이 비어 있으면 헤더 추가
            writer.writerow(['Epoch', 'Layer', 'Sample Index', 'Rank', 'Row Index', 'Channel Index'])
        
        for i, indices in enumerate(top3_indices):
            for rank, (row, col) in enumerate(indices, 1):
                writer.writerow([epoch, layer, i, rank, row, col])




# x = torch.rand(256,197,384)
x = torch.tensor([[[1, 10, 5], 
     [4, 5, 7]]])
process_tensor(x, 15, 'dd')


top3_indices_flat [1 5 2]
channel_dim 3


In [10]:
pip install pandas


     |████████████████████████████████| 12.4 MB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 346 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 508 kB 10.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd

# CSV 파일 경로
file_path = '/home/shkim/QT_DeiT_small/probe_result.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)

# 특정 열 추출 (예: 'Top1 Mean' 열을 추출하는 경우)
top1_mean_column = df['Top1 Mean']

# 여러 열 추출 (예: 'Top1 Mean'과 'Top3 Mean' 열을 동시에 추출하는 경우)
selected_columns = df[['Top1 Mean', 'Top3 Mean']]

# 출력 확인
print(top1_mean_column)
print(selected_columns)


0           10.0
1           10.0
2           10.0
3           10.0
4     ----------
5      Top1 Mean
6           10.0
7           10.0
8           10.0
9           10.0
10    ----------
Name: Top1 Mean, dtype: object
     Top1 Mean          Top3 Mean
0         10.0  7.333333333333333
1         10.0  7.333333333333333
2         10.0  7.333333333333333
3         10.0  7.333333333333333
4   ----------         ----------
5    Top1 Mean          Top3 Mean
6         10.0  7.333333333333333
7         10.0  7.333333333333333
8         10.0  7.333333333333333
9         10.0  7.333333333333333
10  ----------         ----------
